<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/ML_lang_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

Extraction de l'archive téléchargée 

In [9]:
!unzip ./data/data.zip

Archive:  ./data/data.zip
  inflating: data.pkl                


## Chargement du corpus

In [29]:
import pandas as pd


data = pd.read_pickle("./df_corpus.zip") 


data

,content,lang
0,我們試試看！,zho
1,我该去睡觉了。,zho
2,你在干什麼啊？,zho
3,這是什麼啊？,zho
4,今天是６月１８号，也是Muiriel的生日！,zho
...,...,...
1847716,I can't take this anymore.,eng
9199518,That actually was my next question.,eng
307316,I am determined to make a living as a playwright.,eng
5586956,I can't pretend I don't have feelings for you.,eng


# pre-processing

## TODO

In [30]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data['content'], data['lang'], test_size=0.33, random_state=1) 
# ramdom_state = meme corpus (meme decoupage, decouper en 2 train et test le corpus, car on a besoin de faire les tests)

In [31]:
y_train.value_counts()

eng    210888
rus    205654
fra    183456
ita    183158
tur    163044
jpn    146623
por    144717
spa    135491
ber    134416
heb    132591
deu    131979
hun    130613
ukr    119173
kab    116790
nld    103543
fin     93320
pol     77587
zho     56730
dan     35746
swe     31163
ara     28481
ell     23298
fas     16642
bul     16467
hin      9803
kor      6004
lav      2081
gle      1747
hye      1488
swa       400
Name: lang, dtype: int64

# Model

In [32]:
from joblib import Memory
from shutil import rmtree

# Create a temporary folder to store the transformers of the pipeline
mem = Memory(location="/content/cachedir", verbose=0)

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([('TfidfVectorizer', TfidfVectorizer()),
                     ('LinearSVC', LinearSVC()),],verbose=True,memory=mem)


model.fit(x_train, y_train)  




________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(), 5245199                     Мы все это заметили.
976984            Tom is seldom late for school.
7951323                            이게 전환점인 것 같아.
8485234              Ruḥeɣ d asfel ɣef tmurt-iw.
6010768                   Ми чекатимемо на тебе.
                           ...                  
2213971            Бъди много внимателен с това.
8619474     Yanni didn't want to practice Islam.
8441513    Lehdur-is ttḍurrun, dɣa kkren nɣan-t.
2215707                              すぐ行くと約束します。
208440                          すぐに医者を呼んできてください。
Name: content, Length: 2643093, dtype: object, 
5245199    rus
976984     eng
7951323    kor
8485234    ber
6010768    ukr
          ... 
2213971    bul
8619474    eng
8441513    ber
2215707    jpn
208440     jpn
Name: lang, Length: 2643093, dtype: object, 
None, message_clsname='Pipel

/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:355: UserWarning: Persisting input arguments took 33.44s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name],


_______________________________________________fit_transform_one - 64.3s, 1.1min
[Pipeline] ......... (step 2 of 2) Processing LinearSVC, total= 6.7min


Pipeline(memory=Memory(location=/content/cachedir/joblib),
         steps=[('TfidfVectorizer', TfidfVectorizer()),
                ('LinearSVC', LinearSVC())],
         verbose=True)

In [34]:
predictions = model.predict(x_test)

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

In [36]:
#from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ara       1.00      0.96      0.98     13647
         ber       0.90      0.91      0.90     66199
         bul       0.99      0.96      0.98      8054
         dan       0.99      0.99      0.99     17815
         deu       1.00      1.00      1.00     65338
         ell       1.00      0.98      0.99     11493
         eng       1.00      1.00      1.00    103527
         fas       1.00      0.99      0.99      8141
         fin       1.00      0.98      0.99     45547
         fra       1.00      1.00      1.00     89902
         gle       0.99      0.94      0.97       898
         heb       1.00      0.99      1.00     65631
         hin       1.00      0.91      0.95      4756
         hun       1.00      0.97      0.99     64152
         hye       1.00      0.91      0.95       761
         ita       1.00      0.99      0.99     89908
         jpn       0.67      1.00      0.80     72271
         kab       0.90    

In [37]:
metrics.accuracy_score(y_test,predictions)

0.9587969783887506

In [38]:
model.predict(['그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다.'])[0] 

'kor'

In [39]:
model.predict(['ויקי יכולה להיות שיטה טובה לשיתוף ידע בקהילות שפועלות באמצעות האינטרנט או בתוךיות. היא חלופה'])[0]

'heb'

In [40]:
model.predict(['einsetzt. Zudem nutzen auch viele '])[0]

'deu'

In [43]:
model.predict(['الكرام بتحويل الكتابة العربي الى كتابة مفهومة من قبل اغلب برامج التصميم مثل الفوتوشوب و الافترايفكتس و البريمير و الافد ميدا كومبوزر و السموك و برامج اخرى'])[0]

'ara'

In [44]:
model.predict(['से प्रत्येक एक विशिष्ट भाषा से संबंधित है। विकिपीडिया के अलावा, सार्वजनिक और निजी दोनों उपयोग में सैकड़ों हजारों अ'])[0]

'hin'

# Model persistence

In [46]:
from joblib import dump
dump(model, 'lang_detector.joblib') 

['lang_detector.joblib']